In [141]:
import os
import re
import json
import requests
import pandas as pd

In [142]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [143]:
df = pd.read_json(os.path.join('data_raw','wikidata_raw.json'))

In [144]:
vsechny_hodnoty = []
for sloupec in df.columns.to_list():
    sl = df.explode(sloupec)[sloupec].drop_duplicates().to_list()
    for s in sl:
        if re.findall(r'^Q\d\d', str(s)):
            vsechny_hodnoty.append(str(s))
vsechny_hodnoty = list(set(vsechny_hodnoty))

In [145]:
len(vsechny_hodnoty)

203505

In [146]:
kam = 'downloads/wikidata/nazvy'

In [147]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [148]:
nestahovat = []
for f in os.listdir(kam):
    with open(os.path.join(kam, f), "r", encoding="utf-8") as soubor:
        slovnik = json.loads(soubor.read())
    try:
        for q in list(slovnik['entities'].keys()):
            nestahovat.append(q)
    except:
        print(f)
        os.remove(os.path.join(kam, f))

In [149]:
len(nestahovat)

203056

In [150]:
nestahovat = set(nestahovat)

In [151]:
def get_labels(ktere):
    seznam = '|'.join(ktere)
    nazev = '-'.join(ktere[0:3])
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={seznam}&languages=cs|en|sk|de|fr|es|ru&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(kam, f"{nazev}.json"), "w+", encoding="utf-8") as file:
            json.dump(response.json(), file, ensure_ascii=False, indent=4)
        print(f"Data has been saved to {nazev}.json")
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [152]:
stahnout = [x for x in vsechny_hodnoty if x not in nestahovat]

In [153]:
len(stahnout)

450

In [154]:
pracovni = []
pocitadlo = 0
for v in stahnout:
    pocitadlo += 1
    pracovni.append(v)
    if pocitadlo % 50 == 0:
        get_labels(pracovni)
        print(f'{pocitadlo:6}/{len(stahnout)}')
        pracovni = []
get_labels(pracovni)
print('Hotovo!')

Data has been saved to Q75366925-Q113844656-Q172911.json
    50/450
Data has been saved to Q75788932-Q20195276-Q95085538.json
   100/450
Data has been saved to Q7005837-Q134259-Q52727001.json
   150/450
Data has been saved to Q763228-Q77052306-Q20479215.json
   200/450
Data has been saved to Q3098636-Q3510287-Q2060464.json
   250/450
Data has been saved to Q9342744-Q120604573-Q124567270.json
   300/450
Data has been saved to Q3885264-Q5628517-Q928811.json
   350/450
Data has been saved to Q94908440-Q3114808-Q12050111.json
   400/450
Data has been saved to Q1016574-Q1916661-Q47012144.json
   450/450
Data has been saved to .json
Hotovo!
